In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import geodesic
%matplotlib inline

In [2]:
hdb_transport = pd.read_csv('./hdb_ll_mrt_cp.csv',index_col = 0)

In [3]:
hdb_transport.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,HERE_address,Latitude,Longitude,closest_mrt,dist_mrt_km,closest_cp,dist_cp_km
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374,YIO CHU KANG MRT STATION SINGAPORE,1.0909,BLK 173/176 ANG MO KIO AVE 4,0.1152
1,2015-12,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,275000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374,YIO CHU KANG MRT STATION SINGAPORE,1.0909,BLK 173/176 ANG MO KIO AVE 4,0.1152
2,2016-05,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,310000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374,YIO CHU KANG MRT STATION SINGAPORE,1.0909,BLK 173/176 ANG MO KIO AVE 4,0.1152
3,2016-06,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,253000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374,YIO CHU KANG MRT STATION SINGAPORE,1.0909,BLK 173/176 ANG MO KIO AVE 4,0.1152
4,2016-11,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,290000.0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.8374,YIO CHU KANG MRT STATION SINGAPORE,1.0909,BLK 173/176 ANG MO KIO AVE 4,0.1152


In [4]:
#Find the increase in latlong that equates to a 1km difference
amk_ave4_174 = (1.37541, 103.837)
bp_mrt = (1.37541, 103.837+0.009)

print(geodesic(amk_ave4_174,bp_mrt).km)

1.0015886917776877


In [5]:
unique_blocks = hdb_transport[['HERE_address','Latitude','Longitude']].drop_duplicates().copy()

In [6]:
sch_latlong = pd.read_csv('./sch_latlong_corrected.csv',index_col=0)

In [7]:
sch_latlong.head()

,school_name,school_latitude,school_longitude
0,NATIONAL JUNIOR COLLEGE,1.330293,103.804111
1,TEMASEK JUNIOR COLLEGE,1.318493,103.935567
2,HWA CHONG INSTITUTION,1.325889,103.800747
3,BUKIT PANJANG PRIMARY SCHOOL,1.373622,103.769375
4,HAIG GIRLS' SCHOOL,1.382108,103.738613


In [8]:
unique_blocks.tail()

,HERE_address,Latitude,Longitude
79095,782A WOODLANDS CRES Singapore,1.44735,103.80254
79096,117 MARSILING RISE Singapore,1.43836,103.78007
79097,348A YISHUN AVE 11 Singapore,1.42719,103.84126
79098,347B YISHUN AVE 11 Singapore,1.42789,103.84212
79099,348C YISHUN AVE 11 Singapore,1.42697,103.84230


In [9]:
def within_km(block_df,amentities_df,dist):
    """
    This function takes in a DataFrame of the flats, amenities and a given distance.
    It subsets the data before finding the number of amenities within the given distance to be more efficient.
    Saying that an amenity is within x distance is more easily interpreted then trying to explain the subset.
    """
    results = []
    
    def check_amenity(series):
        block = (series[1],series[2])
        number_amenities = 0
        
        #Subset the DataFrame to a square with its length being twice the given distance
        amenities_in_range = amentities_df[(abs(amentities_df.iloc[:,1]-series[1])<=(0.009*dist)) 
                                    & (abs(amentities_df.iloc[:,2]-series[2])<=(0.009*dist))]
        
        #Within the subset check if the amenity is within the given distance 
        for index in range(len(amenities_in_range)):
            amenity = (amenities_in_range.iloc[index][1],amenities_in_range.iloc[index][2])
            
            if geodesic(block,amenity).km < dist:
                number_amenities += 1
                
        #append the number of amenities to the results list
        results.append(number_amenities)
      
    unique_blocks.apply(check_amenity,axis=1)
    
    return results

In [10]:
#Retrieve the counts of schools within 1 and 2km
results1 = within_km(unique_blocks,sch_latlong,1)
results2 = within_km(unique_blocks,sch_latlong,2)

In [11]:
unique_amenities_counts = unique_blocks.copy()

In [12]:
#Append the retrieved results to the DataFrame
unique_amenities_counts['schs_1km'] = results1
unique_amenities_counts['schs_2km'] = results2

In [13]:
unique_amenities_counts.head()

,HERE_address,Latitude,Longitude,schs_1km,schs_2km
0,174 ANG MO KIO AVE 4 Singapore,1.37541,103.83740,6,14
13,541 ANG MO KIO AVE 10 Singapore,1.37412,103.85576,3,11
24,163 ANG MO KIO AVE 4 Singapore,1.37387,103.83834,6,14
33,446 ANG MO KIO AVE 10 Singapore,1.36794,103.85543,5,14
35,557 ANG MO KIO AVE 10 Singapore,1.37145,103.85785,3,12


In [14]:
#Import the amenities geodata
hawker_latlong = pd.read_csv('./hawker_latlong.csv',index_col=0)
sm_latlong = pd.read_csv('./sm_latlong.csv',index_col=0)
sap_sch_latlong = pd.read_csv('./sap_sch_latlong.csv',index_col=0)
autonomous_ind_latlong = pd.read_csv('./autonomous_ind_latlong.csv',index_col=0)
gifted_ind_latlong = pd.read_csv('./gifted_ind_latlong.csv',index_col=0)
ip_ind_latlong = pd.read_csv('./ip_ind_latlong.csv',index_col=0)

In [15]:
#Retrieve the number of amenities within both 1 and 2km
sap1 = within_km(unique_blocks,sap_sch_latlong,1)
sap2 = within_km(unique_blocks,sap_sch_latlong,2)
auto1 = within_km(unique_blocks,autonomous_ind_latlong,1)
auto2 = within_km(unique_blocks,autonomous_ind_latlong,2)
gifted1 = within_km(unique_blocks,gifted_ind_latlong,1)
gifted2 = within_km(unique_blocks,gifted_ind_latlong,2)
ip1 = within_km(unique_blocks,ip_ind_latlong,1)
ip2 = within_km(unique_blocks,ip_ind_latlong,2)

hawker1 = within_km(unique_blocks,hawker_latlong,1)
hawker2 = within_km(unique_blocks,hawker_latlong,2)
sm1 = within_km(unique_blocks,sm_latlong,1)
sm2 = within_km(unique_blocks,sm_latlong,2)

In [16]:
#Append the retrieved results to the DataFrame
unique_amenities_counts['sap_sch_1km'] = sap1
unique_amenities_counts['sap_sch_2km'] = sap2
unique_amenities_counts['autonomous_sch_1km'] = auto1
unique_amenities_counts['autonomous_sch_2km'] = auto2
unique_amenities_counts['gifted_sch_1km'] = gifted1
unique_amenities_counts['gifted_sch_2km'] = gifted2
unique_amenities_counts['ip_sch_1km'] = ip1
unique_amenities_counts['ip_sch_2km'] = ip2

unique_amenities_counts['hawker_1km'] = hawker1
unique_amenities_counts['hawker_2km'] = hawker2
unique_amenities_counts['supermarket_1km'] = sm1
unique_amenities_counts['supermarket_2km'] = sm2

In [17]:
#Take a look at the spread 
unique_amenities_counts.describe()

,Latitude,Longitude,schs_1km,schs_2km,sap_sch_1km,sap_sch_2km,autonomous_sch_1km,autonomous_sch_2km,gifted_sch_1km,gifted_sch_2km,ip_sch_1km,ip_sch_2km,hawker_1km,hawker_2km,supermarket_1km,supermarket_2km
count,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000,8512.000000
mean,1.364969,103.839980,5.546992,15.167646,0.242834,0.774906,0.337406,1.002467,0.097039,0.312030,0.093163,0.369831,1.375117,3.807918,6.258106,18.220042
std,0.041058,0.075056,2.343421,5.115902,0.489949,0.907593,0.562295,1.027466,0.308849,0.608293,0.301003,0.648179,1.642937,3.957905,2.614007,5.647280
min,1.270390,103.685260,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.337998,103.772687,4.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000,14.000000
50%,1.361130,103.845665,6.000000,15.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,6.000000,18.000000
75%,1.388713,103.900020,7.000000,18.000000,0.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000,2.000000,6.000000,8.000000,22.000000
max,1.456910,103.987980,14.000000,30.000000,3.000000,4.000000,3.000000,4.000000,2.000000,3.000000,2.000000,3.000000,9.000000,19.000000,16.000000,39.000000


In [18]:
#Save the DataFrame of amenity counts
unique_amenities_counts.to_csv('unique_amenities_counts.csv')